ALS + GB
Подобрать другие retrivials

In [1]:
%autosave 60
%reload_ext autoreload
%autoreload 2

import sys

sys.path.insert(0, "..")  # noqa: E702

Autosaving every 60 seconds


In [2]:
from catboost.utils import get_gpu_device_count

get_gpu_device_count()

1

In [3]:
import implicit.gpu
import polars as pl
import numpy as np
import implicit
from scipy.sparse import csr_matrix
from pathlib import Path
from catboost import CatBoostRanker, Pool
from datetime import timedelta


implicit.gpu.HAS_CUDA
# from tools.experiment_tracker import ExperimentTracker

/home/laptopml/avito_cup_ml_2025/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

In [4]:
from pathlib import Path

DATA_DIR = Path.cwd().parent / "data" / "avito_ml_cup"
DATA_TEPM_FILE = Path.cwd().parent / "data" / "temp"
# Разделение на три части: тренировочную, валидационную и тестовую
VALID_DAYS_RETRIEVER = 7
VALID_DAYS_RERANKER = 7

In [41]:
# df_test_users = pl.read_parquet(f"{DATA_DIR}/test_users.pq")
# df_clickstream = pl.read_parquet(f"{DATA_DIR}/clickstream.pq")

# df_cat_features = pl.read_parquet(f"{DATA_DIR}/cat_features.pq")
# df_text_features = pl.read_parquet(f"{DATA_DIR}/text_features.pq")
# df_event = pl.read_parquet(f"{DATA_DIR}/events.pq")

In [10]:
def get_cat_and_text_features(data_dir: str):
    df_cat_features = pl.read_parquet(f"{data_dir}/cat_features.pq").lazy()
    df_text_features = pl.read_parquet(f"{data_dir}/text_features.pq").lazy()

    cast_types = {
        "category": pl.Int16,
        "location_mode": pl.Int16,
    }
    for i in range(64):
        cast_types[f"emb_{i}"] = pl.Int8

    df_text_features = df_text_features.with_columns(
        [pl.col("title_projection").arr.get(i).alias(f"t_{i}") for i in range(64)]
    )

    df_cat_features = df_cat_features.join(df_text_features, on="item", how="left")

    agg_exprs = [pl.mean(f"t_{i}").alias(f"emb_{i}") for i in range(64)] + [
        pl.col("location").mode().first().alias("location_mode"),
    ]

    return df_cat_features.group_by(["node", "category"]).agg(agg_exprs).cast(cast_types).collect()

In [5]:
if (data_file := DATA_TEPM_FILE / "cat_and_text_features_preprocess.pq").is_file():
    df_cat_features = pl.read_parquet(data_file)
else:
    df_cat_features = get_cat_and_text_features(DATA_DIR)

In [14]:
# Определяем границы для разделения данных
max_date = df_clickstream["event_date"].max()
retrivial_threshold = max_date - timedelta(
    days=VALID_DAYS_RETRIEVER + VALID_DAYS_RERANKER
)
reranker_threshold = max_date - timedelta(days=VALID_DAYS_RERANKER)

# Разделяем данные
df_train_retrivial = df_clickstream.filter(pl.col("event_date") < retrivial_threshold)
df_train_reranker = df_clickstream.filter(
    pl.col("event_date").is_between(retrivial_threshold, reranker_threshold)
)
df_valid = df_clickstream.filter(pl.col("event_date") > reranker_threshold)


# Создаем оценочные наборы для валидации и тестирования
def prepare_eval_set(df_eval_raw, df_train_data):
    # Выбираем только нужные колонки
    df_eval = df_eval_raw[["cookie", "node", "event"]]

    # Исключаем пары cookie-node, которые уже встречались в тренировочных данных
    df_eval = df_eval.join(df_train_data, on=["cookie", "node"], how="anti")

    # Оставляем только контактные события
    df_eval = df_eval.filter(
        pl.col("event").is_in(
            df_event.filter(pl.col("is_contact") == 1)["event"].unique()
        )
    )

    # Оставляем только существующие в тренировочных данных cookies и nodes
    df_eval = df_eval.filter(
        pl.col("cookie").is_in(df_train_data["cookie"].unique())
    ).filter(pl.col("node").is_in(df_train_data["node"].unique()))

    # Оставляем только уникальные пары cookie-node
    df_eval = df_eval.unique(["cookie", "node"])

    return df_eval


# Подготавливаем валидационный и тестовый наборы
df_valid_eval_retrivial = prepare_eval_set(df_train_reranker, df_train_retrivial)
df_valid_eval_reranker = prepare_eval_set(
    df_valid, pl.concat([df_train_retrivial, df_train_reranker])
)

df_train_retrivial = df_train_retrivial.join(df_event, on="event")
df_train_reranker = df_train_reranker.join(df_event, on="event")

df_valid = df_valid.join(df_event, on="event")

/tmp/ipykernel_6634/4074901764.py:25: DeprecationWarning: `is_in` with a collection of the same datatype is ambiguous and deprecated.
Please use `implode` to return to previous behavior.

See https://github.com/pola-rs/polars/issues/22149 for more information.
  df_eval = df_eval.filter(
/tmp/ipykernel_6634/4074901764.py:32: DeprecationWarning: `is_in` with a collection of the same datatype is ambiguous and deprecated.
Please use `implode` to return to previous behavior.

See https://github.com/pola-rs/polars/issues/22149 for more information.
  df_eval = df_eval.filter(
/tmp/ipykernel_6634/4074901764.py:34: DeprecationWarning: `is_in` with a collection of the same datatype is ambiguous and deprecated.
Please use `implode` to return to previous behavior.

See https://github.com/pola-rs/polars/issues/22149 for more information.
  ).filter(pl.col("node").is_in(df_train_data["node"].unique()))


In [6]:
if (DATA_TEPM_FILE / "train_retrivial.pq").is_file():
    df_train_retrivial = pl.read_parquet(DATA_TEPM_FILE / "train_retrivial.pq")
    df_train_reranker = pl.read_parquet(DATA_TEPM_FILE / "train_reranker.pq")
    df_valid_eval_retrivial = pl.read_parquet(DATA_TEPM_FILE / "valid_eval_retrivial.pq")
    df_valid_eval_reranker = pl.read_parquet(DATA_TEPM_FILE / "valid_eval_reranker.pq")

In [7]:
# Выводим информацию о размерах наборов данных
print(f"Размер тренировочного набора retrivial: {df_train_retrivial.shape}")
print(f"Размер валидационного набора reranker: {df_train_reranker.shape}")
# print(f"Размер валидационного набора: {df_valid.shape}")
print(
    f"Размер валидационного набора для оценки retrivial: {df_valid_eval_retrivial.shape}"
)
print(
    f"Размер валидационного набора для оценки reranker: {df_valid_eval_reranker.shape}"
)

Размер тренировочного набора retrivial: (45631757, 8)
Размер валидационного набора reranker: (11400304, 8)
Размер валидационного набора для оценки retrivial: (72271, 3)
Размер валидационного набора для оценки reranker: (74771, 3)


# Train model

## Train ALS

In [8]:
class ALSRetriever:
    def __init__(self, **params_als):
        self.model = implicit.als.AlternatingLeastSquares(**params_als)

        self._user_id_to_index: dict[int, int] | None = None
        self._item_id_to_index: dict[int, int] | None = None
        self._index_to_item_id: dict[int, int] | None = None
        self._sparse_matrix = None

    def _create_mappings(self, user_ids, item_ids):
        """Создает маппинги между ID и индексами для пользователей и товаров"""
        self._user_id_to_index = {user_id: idx for idx, user_id in enumerate(user_ids)}
        self._item_id_to_index = {item_id: idx for idx, item_id in enumerate(item_ids)}
        self._index_to_item_id = {v: k for k, v in self._item_id_to_index.items()}
        self._index_to_user_id = {v: k for k, v in self._user_id_to_index.items()}

    def fit(
        self,
        users,
        nodes,
        events=None,
        event_weights=None,
        contact_event_boost=10.0,
        contact_events=None,
        is_contact_col=None,
    ):
        """
        Обучает модель ALS

        Args:
            users (pl.Series): ID пользователей
            nodes (pl.Series): ID элементов (товаров)
            events (pl.Series, optional): Типы событий
            event_weights (dict, optional): Словарь весов для разных типов событий
            contact_event_boost (float): Множитель для контактных событий
            contact_events (list, optional): Список ID контактных событий
            is_contact_col (pl.Series, optional): Колонка с флагом контакта

        Returns:
            self: Возвращает self для возможности цепочки вызовов
        """
        user_ids = users.unique().to_list()
        item_ids = nodes.unique().to_list()

        self._create_mappings(user_ids, item_ids)

        rows = users.replace_strict(self._user_id_to_index).to_list()
        cols = nodes.replace_strict(self._item_id_to_index).to_list()

        # Определение весов взаимодействий
        if events is not None and event_weights:
            values = [event_weights.get(event, 1.0) for event in events.to_list()]

            # Если указаны контактные события, увеличиваем их вес
            if contact_events:
                values = [
                    val * contact_event_boost if events[i] in contact_events else val
                    for i, val in enumerate(values)
                ]
        elif is_contact_col is not None:
            # Если есть прямой признак контакта, используем его для весов
            values = [
                contact_event_boost if is_contact else 1.0
                for is_contact in is_contact_col.to_list()
            ]
        else:
            # По умолчанию все взаимодействия имеют вес 1
            values = [1] * len(users)

        # Создаем разреженную матрицу взаимодействий
        self._sparse_matrix = csr_matrix(
            (values, (rows, cols)), shape=(len(user_ids), len(item_ids))
        )

        # Обучаем модель
        self.model.fit(self._sparse_matrix)

    def recommend(self, user_ids, n=40, filter_already_liked_items=True):
        """
        Генерирует рекомендации для указанных пользователей

        Args:
            user_ids (list): Список ID пользователей для рекомендаций
            n (int): Количество рекомендаций для каждого пользователя
            filter_already_liked_items (bool): Исключать ли уже взаимодействовавшие товары
            recalculate_user (bool): Пересчитывать ли модель для пользователя

        Returns:
            pl.DataFrame: DataFrame с рекомендациями
        """
        if not self._user_id_to_index:
            raise ValueError("Сначала надо fit.")
        # Преобразуем ID пользователей в индексы
        user_indices = []
        valid_users = []

        for user_id in user_ids:
            if user_id in self._user_id_to_index:
                user_indices.append(self._user_id_to_index[user_id])  # type: ignore
                valid_users.append(user_id)

        if not user_indices:
            return pl.DataFrame({"cookie": [], "node": [], "scores": []})

        # Получаем рекомендации
        user_index_array = np.array(user_indices)
        recommendations, scores = self.model.recommend(
            user_index_array,
            self._sparse_matrix[user_index_array],  # type: ignore
            N=n,
            filter_already_liked_items=filter_already_liked_items,
        )

        # Преобразуем индексы товаров обратно в ID
        node_lists = [
            [self._index_to_item_id[idx] for idx in rec] for rec in recommendations
        ]  # type: ignore

        # Создаем DataFrame с результатами
        df_pred = pl.DataFrame(
            {"node": node_lists, "cookie": valid_users, "scores": scores.tolist()}
        )

        # Разворачиваем списки
        df_pred = df_pred.explode(["node", "scores"])

        return df_pred

    def save_model(self, path: str):
        import pickle

        model_data = {
            "model": self.model,
            "user_id_to_index": self._user_id_to_index,
            "item_id_to_index": self._item_id_to_index,
            "index_to_item_id": self._index_to_item_id,
            "index_to_user_id": self._index_to_user_id,
            "sparse_matrix": self._sparse_matrix,
        }

        with open(path, "wb") as f:
            pickle.dump(model_data, f)

    @classmethod
    def load(cls, path):
        """
        Загружает модель из файла

        Args:
            path (str): Путь к файлу модели

        Returns:
            ALSRetrivier: Загруженная модель
        """
        import pickle

        with open(path, "rb") as f:
            model_data = pickle.load(f)

        retriever = cls()
        retriever.model = model_data["model"]
        retriever._user_id_to_index = model_data["user_id_to_index"]
        retriever._item_id_to_index = model_data["item_id_to_index"]
        retriever._index_to_item_id = model_data["index_to_item_id"]
        retriever._index_to_user_id = model_data["index_to_user_id"]
        retriever._sparse_matrix = model_data["sparse_matrix"]

        return retriever


def get_als_pred(
    users,
    nodes,
    events,
    user_to_pred,
    event_weights=None,
    generate_candidates: int = 40,
):
    user_ids = users.unique().to_list()
    item_ids = nodes.unique().to_list()

    user_id_to_index = {user_id: idx for idx, user_id in enumerate(user_ids)}
    item_id_to_index = {item_id: idx for idx, item_id in enumerate(item_ids)}
    index_to_item_id = {v: k for k, v in item_id_to_index.items()}

    rows = users.replace_strict(user_id_to_index).to_list()
    cols = nodes.replace_strict(item_id_to_index).to_list()

    # Используем веса событий или 1 по умолчанию
    if event_weights is not None and events is not None:
        values = [event_weights.get(event, 1.0) for event in events.to_list()]
    else:
        values = [1] * len(users)

    sparse_matrix = csr_matrix(
        (values, (rows, cols)), shape=(len(user_ids), len(item_ids))
    )

    model = implicit.als.AlternatingLeastSquares(iterations=10, factors=100)
    model.fit(
        sparse_matrix,
    )

    user4pred = np.array([user_id_to_index[i] for i in user_to_pred])

    recommendations, scores = model.recommend(
        user4pred,
        sparse_matrix[user4pred],
        N=generate_candidates,
        filter_already_liked_items=True,
    )

    df_pred = pl.DataFrame(
        {
            "node": [
                [index_to_item_id[i] for i in i] for i in recommendations.tolist()
            ],
            "cookie": list(user_to_pred),
            "scores": scores.tolist(),
        }
    )
    df_pred = df_pred.explode(["node", "scores"])
    return df_pred

In [9]:
als_retriever = ALSRetriever(
    iterations=50, factors=100, regularization=0.3, use_gpu=True, random_state=123
)
# als_retriever = ALSRetriever(iterations=2, factors=16, regularization=0.3, use_gpu=True, random_state=123)

In [10]:
users = df_train_retrivial["cookie"]
nodes = df_train_retrivial["node"]
events = df_train_retrivial["event"]
eval_users = df_valid_eval_retrivial["cookie"].unique().to_list()

In [11]:
def create_event_weights(clickstream_df: pl.DataFrame):
    event_count = {
        row[0]: row[1] for row in clickstream_df.group_by("event").len().rows()
    }

    return {event_id: 1.0 / np.log1p(count) for event_id, count in event_count.items()}


inverse_freq_weights = create_event_weights(df_train_retrivial.select("event"))

# Можно также попробовать эту альтернативную стратегию взвешивания
# df_pred_alt = get_als_pred(users, nodes, events, eval_users, inverse_freq_weights, 100)

In [12]:
als_retriever.fit(
    users, nodes, events, inverse_freq_weights, 100, list(inverse_freq_weights.keys())
)

100%|██████████| 50/50 [01:24<00:00,  1.69s/it]


In [13]:
df_pred_alt = als_retriever.recommend(eval_users, 500)

In [14]:
del als_retriever

In [15]:
class PopularItemsRetriever:
    """
    Класс для получения популярных товаров по различным критериям
    """

    def __init__(self, clickstream_df=None, categories_df=None):
        """
        Инициализирует ретривер популярных товаров

        Args:
            clickstream_df (pl.DataFrame, optional): DataFrame с данными кликстрима
            categories_df (pl.DataFrame, optional): DataFrame с категориями товаров
        """
        self.clickstream_df = clickstream_df
        self.categories_df = categories_df

        # Кэш для хранения результатов
        self._cache = {}

    def set_data(self, clickstream_df, categories_df=None):
        """
        Устанавливает данные для работы

        Args:
            clickstream_df (pl.DataFrame): DataFrame с данными кликстрима
            categories_df (pl.DataFrame, optional): DataFrame с категориями товаров

        Returns:
            self: Возвращает self для цепочки вызовов
        """
        self.clickstream_df = clickstream_df
        if categories_df is not None:
            self.categories_df = categories_df

        # Сбрасываем кэш при обновлении данных
        self._cache = {}

        return self

    def get_popular_items(self, days=7, top_k=100, use_cache=True):
        """
        Получает популярные товары за указанный период

        Args:
            days (int): Количество дней для анализа
            top_k (int): Количество лучших товаров
            use_cache (bool): Использовать кэш или пересчитать

        Returns:
            pl.DataFrame: Таблица с популярными товарами и количеством взаимодействий
        """
        if self.clickstream_df is None:
            raise ValueError("Не установлены данные кликстрима")

        cache_key = f"popular_{days}_{top_k}"
        if use_cache and cache_key in self._cache:
            return self._cache[cache_key]

        # Определяем временное окно
        cutoff_date = self.clickstream_df["event_date"].max() - timedelta(days=days)
        recent_data = self.clickstream_df.filter(pl.col("event_date") > cutoff_date)

        # Подсчитываем популярность каждого node
        popular_nodes = (
            recent_data.group_by("node")
            .agg(pl.count().alias("interaction_count"))
            .sort("interaction_count", descending=True)
            .head(top_k)
        )

        # Кэшируем результаты
        if use_cache:
            self._cache[cache_key] = popular_nodes

        return popular_nodes

    def get_popular_items_by_category(self, days=7, top_k=20, use_cache=True):
        """
        Получает популярные товары по категориям за указанный период

        Args:
            days (int): Количество дней для анализа
            top_k (int): Количество лучших товаров на категорию
            use_cache (bool): Использовать кэш или пересчитать

        Returns:
            pl.DataFrame: Таблица с популярными товарами по категориям
        """
        if self.clickstream_df is None or self.categories_df is None:
            raise ValueError("Не установлены данные кликстрима или категорий товаров")

        cache_key = f"popular_by_category_{days}_{top_k}"
        if use_cache and cache_key in self._cache:
            return self._cache[cache_key]

        # Определяем временное окно
        cutoff_date = self.clickstream_df["event_date"].max() - timedelta(days=days)
        recent_data = self.clickstream_df.filter(pl.col("event_date") > cutoff_date)

        # Соединяем с категориями
        recent_with_cat = recent_data.join(
            self.categories_df[["node", "category"]].unique(), on="node", how="inner"
        )

        # Подсчитываем популярность по категориям
        popular_by_cat = (
            recent_with_cat.group_by(["category", "node"])
            .agg(pl.count().alias("interaction_count"))
            .sort(["category", "interaction_count"], descending=[False, True])
        )

        # Берем top_k для каждой категории
        result = popular_by_cat.group_by("category").head(top_k)

        # Кэшируем результаты
        if use_cache:
            self._cache[cache_key] = result

        return result

    def get_trending_items(self, days_window=3, previous_days=7, top_k=100):
        """
        Получает тренды - товары, растущие по популярности

        Args:
            days_window (int): Размер окна для текущей популярности (дни)
            previous_days (int): Количество дней для сравнения с предыдущим периодом
            top_k (int): Количество растущих товаров для возврата

        Returns:
            pl.DataFrame: Таблица с растущими товарами
        """
        if self.clickstream_df is None:
            raise ValueError("Не установлены данные кликстрима")

        max_date = self.clickstream_df["event_date"].max()

        # Данные для текущего окна
        current_cutoff = max_date - timedelta(days=days_window)
        current_data = self.clickstream_df.filter(pl.col("event_date") > current_cutoff)

        # Данные для предыдущего окна
        prev_start = current_cutoff - timedelta(days=previous_days)
        prev_data = self.clickstream_df.filter(
            (pl.col("event_date") <= current_cutoff)
            & (pl.col("event_date") > prev_start)
        )

        # Вычисляем популярность для текущего и предыдущего окна
        current_pop = current_data.group_by("node").agg(
            pl.count().alias("current_count")
        )

        prev_pop = prev_data.group_by("node").agg(pl.count().alias("previous_count"))

        # Объединяем и вычисляем рост
        trending = (
            current_pop.join(prev_pop, on="node", how="left")
            .with_columns(
                [
                    pl.col("previous_count").fill_null(0),
                    (pl.col("current_count") - pl.col("previous_count")).alias(
                        "growth"
                    ),
                    (
                        (pl.col("current_count") - pl.col("previous_count"))
                        / (pl.col("previous_count") + 1)
                    ).alias("growth_rate"),
                ]
            )
            .sort("growth_rate", descending=True)
            .head(top_k)
        )

        return trending

    def get_popular_for_user(self, user_id, days=30, top_k=50):
        """
        Получает популярные товары для конкретного пользователя

        Args:
            user_id: ID пользователя (cookie)
            days (int): Количество дней для анализа
            top_k (int): Количество товаров для возврата

        Returns:
            pl.DataFrame: Таблица с товарами, популярными среди похожих пользователей
        """
        if self.clickstream_df is None:
            raise ValueError("Не установлены данные кликстрима")

        # Здесь может быть реализована логика определения похожих пользователей
        # и получения популярных товаров среди них

        # Простая реализация: товары из категорий, с которыми взаимодействовал пользователь
        if self.categories_df is None:
            return self.get_popular_items(days, top_k)

        # Получаем категории товаров, с которыми взаимодействовал пользователь
        user_categories = (
            self.clickstream_df.filter(pl.col("cookie") == user_id)
            .join(
                self.categories_df[["node", "category"]].unique(),
                on="node",
                how="inner",
            )["category"]
            .unique()
        )

        if len(user_categories) == 0:
            return self.get_popular_items(days, top_k)

        # Получаем популярные товары из категорий пользователя
        cutoff_date = self.clickstream_df["event_date"].max() - timedelta(days=days)
        recent_data = self.clickstream_df.filter(pl.col("event_date") > cutoff_date)

        # Фильтруем по категориям пользователя
        user_cat_items = (
            recent_data.join(
                self.categories_df.filter(pl.col("category").is_in(user_categories))[
                    ["node", "category"]
                ].unique(),
                on="node",
                how="inner",
            )
            .group_by("node")
            .agg(pl.count().alias("interaction_count"))
            .sort("interaction_count", descending=True)
            .head(top_k)
        )

        return user_cat_items

In [16]:
popular_retriever = PopularItemsRetriever(df_train_retrivial, df_cat_features)

# Получаем популярные товары за последнюю неделю и месяц
popular_week = popular_retriever.get_popular_items(days=7, top_k=300)
popular_month = popular_retriever.get_popular_items(days=30, top_k=400)
# popular_by_category = popular_retriever.get_popular_items_by_category(days=7, top_k=30)

/tmp/ipykernel_8192/1626469783.py:66: DeprecationWarning: `pl.count()` is deprecated. Please use `pl.len()` instead.
  .agg(pl.count().alias("interaction_count"))


# Двустадийная система рекомендаций
## Этап 1: ALS для первичного отбора релевантных товаров
## Этап 2: CatBoost для ранжирования и уточнения результатов

In [17]:
df_train_reranker

cookie,item,event,event_date,platform,surface,node,is_contact
i64,i64,i64,datetime[ns],i64,i64,u32,i64
1,6830678,17,2025-02-09 13:17:03,2,2,15,0
1,6830678,17,2025-02-09 13:16:58,2,2,15,0
1,14233689,17,2025-02-09 16:53:28,2,2,214458,0
1,7928899,17,2025-02-15 13:44:59,2,11,214231,0
1,18436084,17,2025-02-14 15:14:23,2,11,214458,0
…,…,…,…,…,…,…,…
149989,14257821,17,2025-02-12 12:42:11,2,2,152108,0
149995,16818719,17,2025-02-14 05:59:18,3,11,115733,0
149996,4516135,17,2025-02-09 04:38:46,2,2,122326,0


In [18]:
df_candidates_train = df_pred_alt.join(
    df_train_reranker["node", "cookie", "is_contact"]
    .sort("is_contact", descending=True)
    .unique(["cookie", "node"]),
    on=["cookie", "node"],
    how="left",
).fill_null(0)

In [19]:
from tqdm import tqdm

def improved_frequency_based_negative_sampling(df_candidates, df_reranker_data, negative_ratio=3):
    """
    Улучшенная версия с корректным определением позитивных примеров
    """
    # 1. ТОЛЬКО контактные события как позитивные примеры
    df_positive = df_candidates.join(
        df_reranker_data.filter(pl.col("is_contact") == 1)  # ТОЛЬКО контакты!
        [["cookie", "node"]]
        .unique(["cookie", "node"]),
        on=["cookie", "node"],
        how="inner"
    ).with_columns(pl.lit(1).alias("is_contact"))  # Явно помечаем как позитивные
    
    print(f"Позитивных примеров (только контакты): {df_positive.shape[0]}")
    print(df_positive.columns)
    
    # 2. Все остальные кандидаты - негативные
    df_negative_pool = df_candidates.join(
        df_positive[["cookie", "node"]],
        on=["cookie", "node"],
        how="anti"
    ).with_columns(pl.lit(0).alias("is_contact"))
    
    # 3. Добавляем частоты для взвешенного сэмплирования
    node_frequencies = df_reranker_data.group_by("node").len().rename({"len": "frequency"})
    df_negative_pool = df_negative_pool.join(node_frequencies, on="node", how="left").with_columns(
        pl.col("frequency").fill_null(1)
    )
    
    negative_samples = []
    
    for user in tqdm(df_positive["cookie"].unique()):
        user_positives_count = len(df_positive.filter(pl.col("cookie") == user))
        needed_negatives = user_positives_count * negative_ratio
        
        user_negative_pool = df_negative_pool.filter(pl.col("cookie") == user)
        
        if len(user_negative_pool) == 0:
            continue
            
        # Взвешенное сэмплирование
        user_negative_pool = user_negative_pool.with_columns([
            (pl.col("frequency").log1p() * 0.4 +
             pl.col("scores").rank(descending=True) * 0.6
            ).alias("sampling_weight")
        ])
        
        if len(user_negative_pool) <= needed_negatives:
            user_negatives = user_negative_pool
        else:
            total_weight = user_negative_pool["sampling_weight"].sum()
            sampling_probs = (user_negative_pool["sampling_weight"] / total_weight).to_list()
            
            selected_indices = np.random.choice(
                len(user_negative_pool),
                size=needed_negatives,
                replace=False,
                p=sampling_probs
            )
            
            user_negatives = user_negative_pool[selected_indices]
        
        negative_samples.append(user_negatives.drop(["frequency", "sampling_weight"]))
    
    if negative_samples:
        df_negatives = pl.concat(negative_samples)
        print(df_negatives.columns)
        df_result = pl.concat([df_positive, df_negatives])
    else:
        df_result = df_positive
    
    print(f"Негативных примеров: {len(df_negatives) if negative_samples else 0}")
    print(f"Итоговый размер: {df_result.shape[0]}")
    print(f"Доля позитивных: {df_positive.shape[0] / df_result.shape[0]:.3f}")
    
    return df_result

# Используем улучшенную версию
# df_candidates_train_improved = improved_frequency_based_negative_sampling(
#     df_candidates_train,
#     df_train_reranker,
#     negative_ratio=200
# )

# del df_candidates_train

In [20]:
# Подготовка данных для CatBoost
def prepare_features_for_boosting(
    df_retrieval,
    df_clickstream,
    df_cat_features,
    df_popular_week,
    df_popular_month,
    # df_popular_by_cat,
):
    """Подготавливает признаки для обучения бустинга на основе результатов ALS"""
    max_date = df_clickstream["event_date"].max()

    # Добавляем информацию о рейтинге ALS
    features = df_retrieval.rename({"scores": "als_score"})

    # Добавляем признак о популярности группы товаров
    features = features.join(
        df_popular_week.select(["node", "interaction_count"]).rename(
            {"interaction_count": "popular_week_count"}
        ),
        on="node",
        how="left",
    ).with_columns(pl.col("popular_week_count").fill_null(-1))

    # Добавляем признак о популярности группы товаров
    features = features.join(
        df_popular_month.select(["node", "interaction_count"]).rename(
            {"interaction_count": "popular_month_count"}
        ),
        on="node",
        how="left",
    ).with_columns(pl.col("popular_month_count").fill_null(-1))

    # === ПОЛЬЗОВАТЕЛЬСКИЕ ПРИЗНАКИ ===
    user_features = (
        df_clickstream.group_by("cookie")
        .agg(
            [
                pl.len().alias("user_total_interactions"),
                pl.n_unique("node").alias("user_unique_nodes"),
                pl.n_unique("event").alias("user_unique_events"),
                pl.col("is_contact").sum().alias("user_total_contacts"),
                (pl.col("is_contact").sum() / pl.len()).alias("user_contact_ratio"),
                pl.col("event_date").min().alias("user_first_interaction"),
                pl.col("event_date").max().alias("user_last_interaction"),
            ]
        )
        .with_columns(
            [
                # ОТНОСИТЕЛЬНЫЕ временные признаки
                (pl.col("user_last_interaction") - pl.col("user_first_interaction"))
                .dt.total_days()
                .alias("user_activity_span_days"),
                (max_date - pl.col("user_last_interaction"))
                .dt.total_days()
                .alias("days_since_user_last_activity"),
                (max_date - pl.col("user_first_interaction"))
                .dt.total_days()
                .alias("user_account_age_days"),
                # Интенсивность активности
                # (pl.col("user_total_interactions") /
                #  (pl.col("user_last_interaction") - pl.col("user_first_interaction")).dt.total_days().clip(1, None)
                # ).alias("user_activity_intensity")
            ]
        )
        .drop(["user_first_interaction", "user_last_interaction"])
    )

    features = features.join(user_features, on="cookie", how="left").fill_null(0)
    del user_features

    for days in [3, 7, 14, 30]:
        recent_threshold = max_date - timedelta(days=days)
        recent_user_activity = (
            df_clickstream.filter(pl.col("event_date") > recent_threshold)
            .group_by("cookie")
            .agg(
                [
                    pl.len().alias(f"user_interactions_last_{days}d"),
                    pl.n_unique("node").alias(f"user_unique_nodes_last_{days}d"),
                    pl.col("is_contact").sum().alias(f"user_contacts_last_{days}d"),
                ]
            )
        )
        features = features.join(
            recent_user_activity, on="cookie", how="left"
        ).fill_null(0)
        del recent_user_activity

    item_features = (
        df_clickstream.group_by("node")
        .agg(
            [
                pl.len().alias("item_total_interactions"),
                pl.n_unique("cookie").alias("item_unique_users"),
                pl.col("is_contact").sum().alias("item_total_contacts"),
                (pl.col("is_contact").sum() / pl.len()).alias("item_contact_ratio"),
                pl.col("event_date").min().alias("item_first_seen"),
                pl.col("event_date").max().alias("item_last_seen"),
            ]
        )
        .with_columns(
            [
                # ОТНОСИТЕЛЬНЫЕ временные признаки
                (max_date - pl.col("item_first_seen"))
                .dt.total_days()
                .alias("item_age_days"),
                (max_date - pl.col("item_last_seen"))
                .dt.total_days()
                .alias("days_since_item_last_interaction"),
                # Активный период товара
                # (pl.col("item_last_seen") - pl.col("item_first_seen")).dt.total_days().alias("item_active_period_days")
            ]
        )
        .drop(["item_first_seen", "item_last_seen"])
    )  # Убираем абсолютные даты

    features = features.join(item_features, on="node", how="left").fill_null(0)
    del item_features

    # Добавляем информацию о категории товара
    features = features.join(
        df_cat_features,
        on="node",
        how="left",
    ).fill_null(-1)

    category_features = (
        df_clickstream.join(
            df_cat_features.select(["node", "category"]).unique(), on="node", how="left"
        )
        .group_by("category")
        .agg(
            [
                pl.len().alias("category_total_interactions"),
                pl.n_unique("node").alias("category_unique_items"),
                pl.n_unique("cookie").alias("category_unique_users"),
                pl.col("is_contact").sum().alias("category_total_contacts"),
                (pl.col("is_contact").sum() / pl.len()).alias("category_contact_ratio"),
            ]
        )
    )

    features = features.join(category_features, on=["category"], how="left").fill_null(
        -1
    )
    del category_features

    # === КРОСС-ПРИЗНАКИ ===
    features = features.with_columns(
        [
            # Взаимодействие популярности товара и активности пользователя
            (
                pl.col("item_total_interactions").log1p()
                * pl.col("user_total_interactions").log1p()
            ).alias("item_user_interaction_product"),
            # Соответствие предпочтений пользователя и товара
            (pl.col("user_contact_ratio") * pl.col("item_contact_ratio")).alias(
                "user_item_contact_affinity"
            ),
            # Новизна товара (бинарный признак)
            pl.when(pl.col("item_age_days") < 7)
            .then(1)
            .otherwise(0)
            .alias("is_new_item"),
            # Активный ли пользователь в последнее время
            pl.when(pl.col("days_since_user_last_activity") < 3)
            .then(1)
            .otherwise(0)
            .alias("is_recently_active_user"),
            # Популярный ли товар в категории
            (
                pl.col("item_total_interactions")
                / pl.col("category_total_interactions")
            ).alias("item_category_popularity_ratio"),
            # Специализированность пользователя в категории
            # (pl.col("user_cat_interactions") / pl.col("user_total_interactions")).alias("user_category_specialization")
        ]
    )

    # === ТРЕНДЫ И СЕЗОННОСТЬ ===
    # Рост популярности товара за последнее время
    # recent_popularity = df_clickstream.filter(
    #     pl.col("event_date") > (max_date - timedelta(days=7))
    # ).group_by("node").len().rename({"len": "item_recent_popularity"})

    # older_popularity = df_clickstream.filter(
    #     pl.col("event_date").is_between(
    #         max_date - timedelta(days=14),
    #         max_date - timedelta(days=7)
    #     )
    # ).group_by("node").len().rename({"len": "item_older_popularity"})

    # popularity_trend = recent_popularity.join(older_popularity, on="node", how="outer").with_columns([
    #     pl.col("item_recent_popularity").fill_null(0),
    #     pl.col("item_older_popularity").fill_null(1),  # Избегаем деления на 0
    # ]).with_columns([
    #     # Относительный тренд популярности
    #     (pl.col("item_recent_popularity") / pl.col("item_older_popularity")).alias("item_popularity_trend"),
    #     # Ускорение роста популярности
    #     (pl.col("item_recent_popularity") - pl.col("item_older_popularity")).alias("item_popularity_momentum")
    # ])
    # del recent_popularity, older_popularity

    # features = features.join(
    #     popularity_trend.select(["node", "item_popularity_trend", "item_popularity_momentum"]),
    #     on="node",
    #     how="left"
    # )
    # del popularity_trend

    # === НОРМАЛИЗАЦИЯ НЕКОТОРЫХ ПРИЗНАКОВ ===
    # features = features.with_columns([
    #     # Логарифмическое преобразование для сильно скошенных признаков
    #     pl.col("user_total_interactions").log1p().alias("user_total_interactions_log"),
    #     pl.col("item_total_interactions").log1p().alias("item_total_interactions_log"),
    #     # Z-score нормализация для активности
    #     ((pl.col("user_total_interactions") - pl.col("user_total_interactions").mean()) /
    #      pl.col("user_total_interactions").std()).alias("user_activity_zscore")
    # ])

    return features


# Подготовка данных для обучения CatBoost
def prepare_catboost_data(features_df, df_eval, max_items_per_user=100):
    """Подготавливает данные для обучения CatBoost с позитивными и негативными примерами"""
    # Фильтруем топ-N предсказаний ALS для каждого пользователя
    top_predictions = features_df.group_by("cookie")

    # Помечаем позитивные примеры (те, которые есть в df_eval)
    labeled_data = top_predictions.join(
        df_eval[["cookie", "node"]], on=["cookie", "node"], how="left"
    )

    return labeled_data


# Готовим данные для CatBoost
train_data_for_boosting = prepare_features_for_boosting(
    df_candidates_train,
    df_train_retrivial,
    df_cat_features,
    popular_week,
    popular_month,
    # popular_by_category,
)

In [21]:
train_data_for_boosting = train_data_for_boosting.sort("cookie")
train_data_for_boosting

node,cookie,als_score,is_contact,popular_week_count,popular_month_count,user_total_interactions,user_unique_nodes,user_unique_events,user_total_contacts,user_contact_ratio,user_activity_span_days,days_since_user_last_activity,user_account_age_days,user_interactions_last_3d,user_unique_nodes_last_3d,user_contacts_last_3d,user_interactions_last_7d,user_unique_nodes_last_7d,user_contacts_last_7d,user_interactions_last_14d,user_unique_nodes_last_14d,user_contacts_last_14d,user_interactions_last_30d,user_unique_nodes_last_30d,user_contacts_last_30d,item_total_interactions,item_unique_users,item_total_contacts,item_contact_ratio,item_age_days,days_since_item_last_interaction,category,emb_0,emb_1,emb_2,emb_3,…,emb_38,emb_39,emb_40,emb_41,emb_42,emb_43,emb_44,emb_45,emb_46,emb_47,emb_48,emb_49,emb_50,emb_51,emb_52,emb_53,emb_54,emb_55,emb_56,emb_57,emb_58,emb_59,emb_60,emb_61,emb_62,emb_63,location_mode,category_total_interactions,category_unique_items,category_unique_users,category_total_contacts,category_contact_ratio,item_user_interaction_product,user_item_contact_affinity,is_new_item,is_recently_active_user,item_category_popularity_ratio
i64,i64,f64,i64,i64,i64,i64,i64,i64,i64,f64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,f64,i64,i64,i16,i8,i8,i8,i8,…,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i16,i64,i64,i64,i64,f64,f64,f64,i32,i32,f64
115834,0,1.041707,0,8835,37116,225,73,10,15,0.066667,28,0,28,56,28,6,107,43,10,171,61,13,225,73,15,37116,6409,802,0.021608,29,0,19,-126,94,124,-65,…,-92,5,55,-49,80,-79,83,84,-69,44,-3,-77,36,12,84,-87,114,-123,90,-94,-56,12,97,106,17,30,2348,2173922,4150,54731,50632,0.023291,57.03395,0.001441,0,1,0.017073
115686,0,0.936484,0,-1,-1,225,73,10,15,0.066667,28,0,28,56,28,6,107,43,10,171,61,13,225,73,15,9211,3749,189,0.020519,29,0,19,-123,116,126,-48,…,-65,5,26,-33,0,-37,58,53,-78,-1,37,-27,-8,-9,92,-72,80,-111,58,-70,-32,-10,70,83,17,-4,2348,2173922,4150,54731,50632,0.023291,49.480065,0.001368,0,1,0.004237
116123,0,0.930687,0,8776,34225,225,73,10,15,0.066667,28,0,28,56,28,6,107,43,10,171,61,13,225,73,15,34225,7229,611,0.017852,29,0,19,-119,101,125,-52,…,-77,25,76,-93,8,18,6,100,-90,-11,50,-86,48,-49,24,-80,10,-109,4,-24,-80,4,94,91,-15,37,2348,2173922,4150,54731,50632,0.023291,56.594401,0.00119,0,1,0.015743
214236,0,0.929946,0,-1,-1,225,73,10,15,0.066667,28,0,28,56,28,6,107,43,10,171,61,13,225,73,15,13252,4651,631,0.047615,29,0,37,-124,97,123,-37,…,-68,-47,69,-16,-2,0,59,99,-83,-34,45,-8,16,-3,88,-62,61,-106,30,-90,0,2,96,89,64,10,2348,1873464,258,81384,120806,0.064483,51.451606,0.003174,0,1,0.007074
115715,0,0.917505,0,-1,-1,225,73,10,15,0.066667,28,0,28,56,28,6,107,43,10,171,61,13,225,73,15,14866,5351,324,0.021795,29,0,19,-122,121,125,-70,…,-35,25,68,-44,-39,-44,89,86,-110,-42,64,18,-31,-1,72,-79,91,-123,66,-14,-64,0,71,102,40,31,2348,2173922,4150,54731,50632,0.023291,52.074534,0.001453,0,1,0.006838
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
382938,149998,0.163314,0,-1,-1,308,62,5,4,0.012987,27,1,29,5,5,0,20,9,1,110,25,2,308,62,4,780,461,21,0.026923,29,0,51,-74,-124,112,32,…,-105,32,34,-83,-19,-38,94,25,-110,-69,58,-59,-2,1,53,-95,60,-76,46,-79,-24,1,111,113,48,-42,1540,3946889,113371,52231,126682,0.032097,38.18735,0.00035,0,1,0.000198
152776,149998,0.162195,0,-1,-1,308,62,5,4,0.012987,27,1,29,5,5,0,20,9,1,110,25,2,308,62,4,364,205,10,0.027473,29,0,40,115,-128,81,-128,…,-128,23,123,-61,15,-22,29,118,-98,-49,65,-127,30,51,102,-27,34,-108,0,-108,-1,69,126,115,-29,-53,8383,5897889,2061,58039,70364,0.01193,33.826125,0.000357,0,1,0.000062
151689,149998,0.161612,0,-1,-1,308,62,5,4,0.012987,27,1,29,5,5,0,20,9,1,110,25,2,308,62,4,233,135,0,0.0,29,0,40,127,-128,34,-128,…,-128,47,89,-58,21,-80,67,82,-111,-2,76,-107,-36,6,99,-88,69,-53,32,-125,14,65,110,117,2,55,7950,5897889,2061,58039,70364,0.01193,31.277218,0.0,0,1,0.00004


In [22]:
train_data_for_boosting = train_data_for_boosting.cast(
    {"als_score": pl.Float32,
     "category_contact_ratio": pl.Float32,
     "item_user_interaction_product": pl.Float32,
     "user_item_contact_affinity": pl.Float32,
     "item_category_popularity_ratio": pl.Float32}
)

In [5]:
# train_data_for_boosting.write_parquet(DATA_TEPM_FILE / "train_data_for_boosting.pq")
train_data_for_boosting = pl.read_parquet(DATA_TEPM_FILE / "train_data_for_boosting.pq")

In [6]:
feature_cols = train_data_for_boosting.columns
feature_cols.remove("is_contact")
feature_cols.remove("cookie")

target_col = "is_contact"

cat_cols = ["category", "location_mode"]

In [7]:
train_data_boosting = Pool(train_data_for_boosting.select(feature_cols).to_pandas(), train_data_for_boosting.select(target_col).to_pandas(), cat_features=cat_cols, group_id=train_data_for_boosting["cookie"].to_list())

In [8]:
model = CatBoostRanker(
    boosting_type="Plain",
    iterations=50,
    learning_rate=0.05,
    depth=8,
    objective="PairLogitPairwise:max_pairs=100000",
    random_seed=42,
    verbose=5,
    nan_mode="Min",
    task_type="CPU",
    border_count=32
)

In [9]:
model.fit(train_data_boosting)

Pairwise scoring loss functions on CPU do not support one hot features. OneHotMaxSize set to 1
0:	learn: 0.6725297	total: 1m 5s	remaining: 53m 5s
5:	learn: 0.5992890	total: 7m 5s	remaining: 52m 2s
10:	learn: 0.5576937	total: 13m 3s	remaining: 46m 17s


KeyboardInterrupt: 

# Оценка моделей
## Метрика Recall@K

In [26]:
def recall_at(df_true, df_pred, k=40):
    """Вычисляет метрику Recall@K для рекомендаций"""
    return (
        df_true[["node", "cookie"]]
        .join(
            df_pred.group_by("cookie")
            .head(k)
            .with_columns(value=1)[["node", "cookie", "value"]],
            how="left",
            on=["cookie", "node"],
        )
        .select([pl.col("value").fill_null(0), "cookie"])
        .group_by("cookie")
        .agg([pl.col("value").sum() / pl.col("value").count()])["value"]
        .mean()
    )


def evaluate_models(
    df_true, df_als_pred, df_catboost_pred=None, k_values=[10, 20, 30, 40]
):
    """Сравнивает модели по метрике Recall@K для разных значений K"""
    results = {}

    for k in k_values:
        results[f"ALS_recall@{k}"] = recall_at(df_true, df_als_pred, k=k)
        if df_catboost_pred is not None:
            results[f"CatBoost_recall@{k}"] = recall_at(df_true, df_catboost_pred, k=k)

    return results


# Оцениваем модель ALS на валидационном наборе
als_results = evaluate_models(
    df_valid_eval_retrivial, df_pred_alt, k_values=[10, 20, 30, 40]
)
print("\nРезультаты ALS модели на валидационном наборе:")
for metric, value in als_results.items():
    print(f"{metric}: {value:.4f}")



Результаты ALS модели на валидационном наборе:
ALS_recall@10: 0.0652
ALS_recall@20: 0.1030
ALS_recall@30: 0.1320
ALS_recall@40: 0.1569


In [27]:
# recall_at(df_eval, df_pred_alt, k=40)

# Можно сохранить обученные модели и результаты для дальнейшего использования

# Пример организации модулей для ClearML:
"""
from tools.experiment_tracker import ExperimentTracker

# Инициализация трекера экспериментов
tracker = ExperimentTracker(project_name="avito_cup_ml_2025", task_name="als_catboost_pipeline")

# Логирование параметров
tracker.log_params({
    'als_factors': 60,
    'als_iterations': 10,
    'catboost_iterations': 500,
    'catboost_learning_rate': 0.05,
    'catboost_depth': 6
})

# Логирование метрик
tracker.log_metrics(als_results)

# Сохранение моделей
tracker.save_model(model, 'catboost_ranker')

# Завершение эксперимента
tracker.close()
"""


# df_train_reranker = df_train_reranker.drop("is_contact")
df_train_retriever_and_reranker = pl.concat([df_train_retrivial, df_train_reranker])

# Применение двустадийной модели на тестовом наборе
# Сначала получаем рекомендации от ALS

if not df_valid_eval_reranker.is_empty():
    test_users = df_valid_eval_reranker["cookie"].unique().to_list()
else:
    # Для тестовых пользователей
    test_users = df_test_users["cookie"].unique().to_list()


# Объединяем тренировочные и валидационные данные для финального обучения
full_train_users = df_train_retriever_and_reranker["cookie"]
full_train_nodes = df_train_retriever_and_reranker["node"]
full_train_events = df_train_retriever_and_reranker["event"]

# Создаем веса на основе обратной частоты и контактности
inverse_freq_weights = create_event_weights(
    df_train_retriever_and_reranker.select("event")
)

In [28]:
als_retriever = ALSRetriever(
    iterations=50, factors=100, regularization=0.3, random_state=123
)

als_retriever.fit(
    full_train_users,
    full_train_nodes,
    full_train_events,
    inverse_freq_weights,
    100,
    list(inverse_freq_weights.keys()),
)

100%|██████████| 50/50 [01:42<00:00,  2.05s/it]


In [29]:
# Получаем предсказания ALS для тестового набора
test_als_pred = als_retriever.recommend(test_users, 700)

In [30]:
popular_retriever = PopularItemsRetriever(
    df_train_retriever_and_reranker, df_cat_features
)

# Получаем популярные товары за последнюю неделю и месяц
popular_week = popular_retriever.get_popular_items(days=7, top_k=300)
popular_month = popular_retriever.get_popular_items(days=30, top_k=300)
# popular_by_category = popular_retriever.get_popular_items_by_category(days=7, top_k=30)

/tmp/ipykernel_3429/1626469783.py:66: DeprecationWarning: `pl.count()` is deprecated. Please use `pl.len()` instead.
  .agg(pl.count().alias("interaction_count"))


In [31]:
test_data_for_boosting = prepare_features_for_boosting(
    test_als_pred,
    df_train_retriever_and_reranker,
    df_cat_features,
    popular_week,
    popular_month,
    # popular_by_category,
)

In [32]:
test_data_for_boosting = test_data_for_boosting.sort("cookie")

test_data_for_boosting = test_data_for_boosting.cast(
    {"als_score": pl.Float32,
     "category_contact_ratio": pl.Float32,
     "item_user_interaction_product": pl.Float32,
     "user_item_contact_affinity": pl.Float32,
     "item_category_popularity_ratio": pl.Float32}
)

test_data_for_boosting

node,cookie,als_score,popular_week_count,popular_month_count,user_total_interactions,user_unique_nodes,user_unique_events,user_total_contacts,user_contact_ratio,user_activity_span_days,days_since_user_last_activity,user_account_age_days,user_interactions_last_3d,user_unique_nodes_last_3d,user_contacts_last_3d,user_interactions_last_7d,user_unique_nodes_last_7d,user_contacts_last_7d,user_interactions_last_14d,user_unique_nodes_last_14d,user_contacts_last_14d,user_interactions_last_30d,user_unique_nodes_last_30d,user_contacts_last_30d,item_total_interactions,item_unique_users,item_total_contacts,item_contact_ratio,item_age_days,days_since_item_last_interaction,category,emb_0,emb_1,emb_2,emb_3,emb_4,…,emb_38,emb_39,emb_40,emb_41,emb_42,emb_43,emb_44,emb_45,emb_46,emb_47,emb_48,emb_49,emb_50,emb_51,emb_52,emb_53,emb_54,emb_55,emb_56,emb_57,emb_58,emb_59,emb_60,emb_61,emb_62,emb_63,location_mode,category_total_interactions,category_unique_items,category_unique_users,category_total_contacts,category_contact_ratio,item_user_interaction_product,user_item_contact_affinity,is_new_item,is_recently_active_user,item_category_popularity_ratio
i64,i64,f32,i64,i64,i64,i64,i64,i64,f64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,f64,i64,i64,i16,i8,i8,i8,i8,i8,…,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i16,i64,i64,i64,i64,f32,f32,f32,i32,i32,f32
115834,0,1.184847,8004,37031,258,83,10,16,0.062016,33,2,35,14,8,1,33,16,1,140,55,11,244,80,16,45120,7435,994,0.02203,36,0,19,-126,94,124,-65,3,…,-92,5,55,-49,80,-79,83,84,-69,44,-3,-77,36,12,84,-87,114,-123,90,-94,-56,12,97,106,17,30,2348,2715706,4319,59853,64080,0.023596,59.553101,0.001366,0,1,0.016614
115669,0,0.949044,-1,-1,258,83,10,16,0.062016,33,2,35,14,8,1,33,16,1,140,55,11,244,80,16,10430,4391,253,0.024257,36,0,19,-126,90,120,-34,0,…,-72,0,43,-30,14,-12,41,81,-79,-22,58,-43,19,-15,65,-49,62,-93,54,-52,-31,3,79,87,20,6,2348,2715706,4319,59853,64080,0.023596,51.414761,0.001504,0,1,0.003841
115705,0,0.941426,-1,-1,258,83,10,16,0.062016,33,2,35,14,8,1,33,16,1,140,55,11,244,80,16,14651,3830,296,0.020203,36,0,19,-126,108,123,-50,21,…,-84,-59,35,-44,64,-30,16,102,-99,-18,67,-37,-19,11,89,-82,95,-114,67,-55,-80,8,75,87,13,0,2348,2715706,4319,59853,64080,0.023596,53.30294,0.001253,0,1,0.005395
214236,0,0.930697,-1,-1,258,83,10,16,0.062016,33,2,35,14,8,1,33,16,1,140,55,11,244,80,16,16719,5589,830,0.049644,36,0,37,-124,97,123,-37,-30,…,-68,-47,69,-16,-2,0,59,99,-83,-34,45,-8,16,-3,88,-62,61,-106,30,-90,0,2,96,89,64,10,2348,2337661,258,87615,151546,0.064828,54.036602,0.003079,0,1,0.007152
116118,0,0.910173,37579,155099,258,83,10,16,0.062016,33,2,35,14,8,1,33,16,1,140,55,11,244,80,16,190813,16154,4239,0.022215,36,0,19,-120,96,126,-45,-85,…,-92,50,84,-88,19,7,-5,90,-92,-4,74,-97,24,-44,34,-88,18,-111,15,-9,-81,11,87,95,-3,42,2348,2715706,4319,59853,64080,0.023596,67.565773,0.001378,0,1,0.070263
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
141694,149997,0.246805,-1,-1,284,117,10,15,0.052817,35,1,36,79,21,11,117,33,14,118,33,14,199,74,15,440,213,4,0.009091,36,0,24,-93,-105,127,-31,67,…,-93,0,114,-101,18,-29,76,88,-100,68,27,78,16,-30,75,-28,69,-109,7,-47,-25,-43,93,59,-85,-6,2348,2503110,18943,56187,78336,0.031295,34.418259,0.00048,0,1,0.000176
91082,149997,0.24679,-1,-1,284,117,10,15,0.052817,35,1,36,79,21,11,117,33,14,118,33,14,199,74,15,661,307,7,0.01059,36,0,33,-126,6,125,12,-84,…,-83,20,27,-104,-94,-45,95,101,-123,-31,46,-6,-30,-37,-23,-87,49,-100,88,-85,-33,5,45,29,12,-61,5651,834553,17726,20880,9751,0.011684,36.714417,0.000559,0,1,0.000792
177653,149997,0.246746,-1,-1,284,117,10,15,0.052817,35,1,36,79,21,11,117,33,14,118,33,14,199,74,15,206,106,2,0.009709,36,0,32,-86,-128,127,-93,120,…,-43,47,16,-120,43,-31,71,121,-110,-82,125,16,11,24,55,-128,0,-128,57,-37,-30,115,53,114,-71,44,340,861823,14572,15947,10176,0.011808,30.143135,0.000513,0,1

In [33]:
df_pred_catboost = (
    pl.DataFrame(
        {
            "node": test_data_for_boosting["node"],
            "cookie": test_data_for_boosting["cookie"],
            "catboost_score": model.predict(test_data_for_boosting.to_pandas()),
        }
    )
    .sort(by=["cookie", "catboost_score"], descending=True)
    .group_by("cookie")
    .head(200)
)

In [37]:
# df_pred_catboost[["cookie", "node"]].write_csv("../data/predict/baseline_als_catboost_add_new_features_v1.0.csv")

In [42]:
sorted(df_test_users["cookie"].unique().to_list()) == sorted(test_users)

False

In [34]:
df_valid_eval_reranker

cookie,node,event
i64,u32,i64
75221,194689,4
31901,130860,10
12649,243181,5
147340,200840,4
56128,351794,10
…,…,…
102904,243205,19
106527,214268,10
33089,71549,19


In [36]:
test_als_results = evaluate_models(
    df_valid_eval_reranker, test_als_pred, df_pred_catboost, k_values=[10, 20, 30, 40]
)
print("\nРезультаты ALS модели на тестовом наборе:")
for metric, value in test_als_results.items():
    print(f"{metric}: {value:.4f}")


Результаты ALS модели на тестовом наборе:
ALS_recall@10: 0.0641
CatBoost_recall@10: 0.0724
ALS_recall@20: 0.1006
CatBoost_recall@20: 0.1126
ALS_recall@30: 0.1290
CatBoost_recall@30: 0.1452
ALS_recall@40: 0.1529
CatBoost_recall@40: 0.1721


----

In [ ]:
test_als_results = evaluate_models(
    df_valid_eval_reranker,
    test_als_pred,
    df_pred_catboost,
    k_values=[10, 20, 30, 40, 50, 60, 70, 100, 150],
)
print("\nРезультаты ALS модели на тестовом наборе:")
for metric, value in test_als_results.items():
    print(f"{metric}: {value:.4f}")


Результаты ALS модели на тестовом наборе:
ALS_recall@10: 0.0641
CatBoost_recall@10: 0.0740
ALS_recall@20: 0.1007
CatBoost_recall@20: 0.1153
ALS_recall@30: 0.1290
CatBoost_recall@30: 0.1479
ALS_recall@40: 0.1529
CatBoost_recall@40: 0.1751
ALS_recall@50: 0.1733
CatBoost_recall@50: 0.1985
ALS_recall@60: 0.1909
CatBoost_recall@60: 0.2179
ALS_recall@70: 0.2077
CatBoost_recall@70: 0.2339
ALS_recall@100: 0.2497
CatBoost_recall@100: 0.2785
ALS_recall@150: 0.3031
CatBoost_recall@150: 0.3327
